In [6]:
import numpy as np
from ezc3d import c3d
from pyomeca import Markers
import importlib
import os
import glob
from pathlib import Path
import dataprocesing as dp

In [15]:
importlib.reload(dp)

for path in Path(r'C:\Users\Grafika Multimedia\Desktop\Miecze_Praktyki\baza').rglob('*.c3d'):
    czas = dp.read_labels(str(path),200)
    temp = []
    for i in range(len(czas[0])):
        if czas[0][i] != 0 and czas[1][i] != 0:
            temp.append([int(czas[0][i]), int(czas[1][i])])
            
    dp.cropp_c3dfile(temp, str(path))